In [1]:
import torch

In [ ]:
# incorporating more context into the data. 

In [4]:
raw = open("data/names.txt").read()
names = raw.split("\n")

32033

In [22]:
chars = sorted(list(set("".join(names) + ".")))
char_to_num = { char: num for num, char in enumerate(chars) }
num_to_char = { num: char for char, num in char_to_num.items() }

In [10]:
context_window = 3
for name in names:
  name = ("." * (context_window - 1)) + name + "."
  for ngram_end_idx in range(context_window - 1, len(name)):
    n_gram = name[ngram_end_idx - 2: ngram_end_idx + 1]
    print(n_gram)
  print(name)
  break

..e
.em
emm
mma
ma.
..emma.


In [28]:
torch.eye(5)[[0, 1, 2, 3], [0, 1, 2, 3]]

tensor([1., 1., 1., 1.])

In [37]:
X = []
Y = []
eye = torch.eye(len(chars))

context_window = 3 # doesn't include output i guess.
for name in names:
  string = "." * (context_window) + name + "."
  string_nums = [char_to_num[char] for char in string]

  for idx in range(context_window, len(string)):
    substring_nums = string_nums[idx - context_window:idx]
    target_num = string_nums[idx]
    x = eye[substring_nums]; X.append(x)
    y = eye[target_num]; Y.append(y)
    
    # print(f"{string[idx - context_window:idx]} -> {string[idx]}")
    # print(f"{substring_nums} -> {target_num}")
    # print(x, y)

X = torch.stack(X) # stack to merge list of tensors
Y = torch.stack(Y)

X.shape, Y.shape

(torch.Size([228146, 3, 27]), torch.Size([228146, 27]))

In [55]:
# have an embedding matrix for the inputs. 
# for each example, want to pass the 3 27 dimensional vectors via matrix multiplication to pluck out the corresponding 2 dimensional vector. 
generator = torch.Generator().manual_seed(14)
embedding_vector_dimensionality = 2
embedding_matrix = torch.randn((len(chars), embedding_vector_dimensionality), generator=generator, requires_grad=True) # all operation results involving this tensor will have gradients tracked. 
# torch.eye(27)[5].view(1, -1) @ embedding_matrix # [ 1.8372, -1.4072]

# forward pass
embeddings = X @ embedding_matrix
embeddings.shape

# c-style indexing. will go from the rightmost dimension, and increment gradually. 
# view works here. when building out that group of 6, will go 2 + 2 + 2 -> 6. order is intrinsically maintained through position within that group of 6. 
embedding_layer_size = context_window * embedding_vector_dimensionality
merged_embedding = embeddings.view(-1, embedding_layer_size)
# merged_embedding.shape

# not sure what the size for the hidden layer should be.  
hidden_layer_size = 100
hidden_weights = torch.randn((embedding_layer_size, hidden_layer_size), generator=generator, requires_grad=True) # dot product to generate output for a node in hidden layer. 6 inputs, 6 weights, for a single output. need to repeat this for each node. (, embed_size) @ (embed_size, hidden_size). matrix multiplication as a conveneint means to perform this computation. 
hidden_biases = torch.randn((hidden_layer_size), generator=generator, requires_grad=True)

hidden_preactivation = merged_embedding @ hidden_weights + hidden_biases
hidden_activation = torch.maximum(hidden_preactivation, torch.ones_like(hidden_preactivation)) # maximum() does element-wise max - basically relu relu

output_layer_size = len(chars)
output_weights = torch.randn((hidden_layer_size, output_layer_size), generator=generator, requires_grad=True) # for a given example, dot product between hidden ouput and output weights for each neuron in output layer.
output_biases = torch.randn((output_layer_size), generator=generator, requires_grad=True)

output_preactivation = hidden_activation @ output_weights + output_biases
output_activation = output_preactivation # no activation function for output layer.
logits = output_activation

counts = logits.exp()
probabilities = counts / counts.sum(dim=1, keepdim=True)

# hidden_activation.shape
# output_activation.shape
probabilities.shape

torch.Size([228146, 27])

torch.Size([228146, 6])

In [ ]:
# not quite what is wanted. 
# anyhow, the goal this time around is to incorporate more context into the function. 